In [2]:
from pyfaidx import Fasta
from pybedtools import BedTool

In [ ]:
ls ~/Dropbox/kiwi/colour/Myb210/Annotations/Chr9_Myb210.recode.nomissing.vcf.gz

In [3]:
!cat  ./test/test-data/Chr9_Myb210.vcf | awk '!/##/' | head -n3

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	16-2-1A	16-1-1A	39-4-5B	39-4-6B	Hort16A	CK15_02	CK51-11-KK14-03-06A	CK51-09-KK03-07-09B	CK51-06-KK03-07-13A	52-13-22D-KK14-07-02A	CK51_05
Chr9	1390048	.	ATAT	ATAA,ATTT	.	PASS	AR2=0.694;DR2=0.755;AF=0.282,0.085	GT:DS:GP	0|0:0.366:0.634,0.295,0,0.07,0,0	0|0:0.366:0.634,0.295,0,0.07,0,0	0|0:0.236:0.764,0.191,0,0.046,0,0	0|1:1.365:0.001,0.634,0.295,0,0.07,0	2|2:1.743:0,0,0,0.257,0.12,0.623	0|1:1:0,1,0,0,0,0	0|1:1:0,1,0,0,0,0	0|0:0:1,0,0,0,0,0	0|1:1:0,1,0,0,0,0	0|1:1:0,1,0,0,0,0	0|0:0:1,0,0,0,0,0
Chr9	1390058	.	A	T	.	PASS	AR2=0.924;DR2=0.926;AF=0.101	GT:DS:GP	0|0:0.059:0.941,0.059,0	0|0:0.059:0.941,0.059,0	0|0:0.059:0.941,0.059,0	0|0:0.059:0.941,0.059,0	1|1:1.969:0,0.031,0.969	0|0:0:1,0,0	0|0:0.005:0.995,0.005,0	0|0:0:1,0,0	0|0:0:1,0,0	0|0:0.018:0.982,0.018,0	0|0:0:1,0,0
awk: write error on /dev/stdout
 input record number 146, file 
 source line number 1
cat: stdout: Broken pipe


In [5]:
myfa= Fasta("/Users/johnmccallum/Documents/Kiwifruit_pseudomolecule.fa")
mybed=BedTool("./test/test-data/Chr9_Myb210.vcf")
mybed

<BedTool(./test/test-data/Chr9_Myb210.vcf)>

In [6]:
mybedtbx=mybed.tabix(force=True)
mybedtbx

<BedTool(./test/test-data/Chr9_Myb210.vcf.gz)>

In [11]:
%%timeit
[X for X in mybed.intervals]

100 loops, best of 3: 13.1 ms per loop


In [12]:
%%timeit
[X for X in mybedtbx.intervals]

100 loops, best of 3: 15.4 ms per loop


In [ ]:
mybedtbx[10].fields

In [ ]:
!grep '>' /Users/johnmccallum/Documents/Kiwifruit_pseudomolecule.fa  | head -n 5

In [ ]:
my_int=mybedtbx.tabix_intervals('A9scaffold929:5800-6000')
print(my_int)

In [ ]:
[(X.start,X.length) for X in mybedtbx.tabix_intervals('A9scaffold929:5800-6000') ]

In [ ]:
mybedtbx[0]

In [ ]:
my_target=BedTool('A9scaffold929 5884 5885 ',from_string= True)
print(my_target)

In [ ]:
my_target[0]

In [ ]:
myfa.faidx.filename.replace("fa","fa.fai")

In [ ]:
print(my_int.intersect(my_target.slop(b=100,g=myfa.faidx.filename.replace("fa","fa.fai"))))

In [68]:
class VcfPrimerDesign:
    """A primer design object that is primed
    with genome reference and vcf variant data
    """
    def __init__(self,reference,vcf_file,desc):
        """
        Usage:  PrimerDesign(reference, vcf.gz, description)
        Initialise a design object witha  reference assembly and
        annotation file(s)
        """
        self.reference = Fasta(reference)
        self.annotation=BedTool(vcf_file).tabix(force=True)
        self.desc=desc
        self.genome=self.reference.filename.replace("fa","fa.fai")
    
    
    def getseqslicedict(self,target,max_size):
        """Pass a target to a designer and get a dictionary
        slice that we can pass to P3
        """
        target_int=target.slop(b=max_size,g=self.genome)
        offset=target_int[0].start
        sldic=dict(SEQUENCE_ID=self.desc)
        sldic['TARGET_ID']=str(target[0].chrom+ "_" + str(target_int[0].start) +"_" + str(target_int[0].end))
        sldic['SEQUENCE_TEMPLATE']=str(self.reference[target[0].chrom][target_int[0].start:target_int[0].end].seq)
        #slice_annot=[(X.start -offset,X.length) for X in (self.annotations - target) if (X.chrom==target[0].chrom) & \
         #           (X.start > target_int[0].start) & (X.end < target_int[0].end)]
        slice_annot=self.
        sldic['SEQUENCE_EXCLUDED_REGION']=slice_annot
        sldic['SEQUENCE_TARGET']= (target[0].start -offset,target[0].length)
        return sldic



-----------------

Try Out 

In [69]:
mybdesigner=VcfPrimerDesign('./test/test-data/Kiwifruit_pseudomolecule.fa','./test/test-data/Chr9_Myb210.vcf','Myb')

In [70]:
my_target=BedTool('Chr9 1391006 1391007',from_string=True)
my_target_window=my_target.slop(b=100,g='./test/test-data/Kiwifruit_pseudomolecule.fa.fai')
mybdesigner.getseqslicedict(my_target,200)

{'SEQUENCE_EXCLUDED_REGION': '',
 'SEQUENCE_ID': 'Myb',
 'SEQUENCE_TARGET': (200, 1),
 'SEQUENCE_TEMPLATE': 'TGAAGTTTTTTGTATAACTTTTCTAGAAAAATTCCATTTTTCTATTGAGCAAGGCAGACTTGACCGGAAAAATATTTAAATGGGCCGTTCAACTGGTAGACTTCTGCTTTAAGTACCGGTCTCCCACAGCCATTAAAGCCCAAGTATTTACTGATTTCATGGGGCTTTGATGTGTGTTGATAAGAGTTGATAAGACATTAAGGGTTGCTTGTTAACGGTTTAATGTATATTTTTTTGTTTAATTTTTATTTTTAAAAATAAATATAAATATAAATTTATTTGTGTTGTAAAAAATAAATGAAAATAAGGCTAAACAACTTTCACTATTTTTGAAAATAGTGAAAACAAGAAAAAATTATTTTTGTTTTGTTTATAAAATTTTTTCATTCAAAATTTTTTTA',
 'TARGET_ID': 'Chr9_1390806_1391207'}

In [71]:
[X for X in my_target_window.intervals][0]

Interval(Chr9:1390906-1391107)

In [72]:
[(X.start,X.length) for X in mybdesigner.annotation.tabix_intervals([X for X in my_target_window.intervals][0])]

[(1390912, 1),
 (1390944, 1),
 (1390954, 1),
 (1390956, 1),
 (1390957, 1),
 (1390960, 1),
 (1390961, 1),
 (1390963, 1),
 (1390964, 1),
 (1390965, 1),
 (1390966, 1),
 (1390967, 1),
 (1390968, 1),
 (1390969, 1),
 (1390970, 1),
 (1390971, 1),
 (1390972, 1),
 (1390973, 1),
 (1390974, 1),
 (1390975, 1),
 (1390976, 1),
 (1390977, 1),
 (1390978, 1),
 (1390979, 1),
 (1390980, 1),
 (1390981, 1),
 (1390982, 1),
 (1390983, 1),
 (1390984, 1),
 (1390985, 1),
 (1390986, 1),
 (1390987, 1),
 (1390988, 1),
 (1390989, 1),
 (1390990, 1),
 (1390992, 1),
 (1390993, 1),
 (1390995, 1),
 (1390999, 1),
 (1391002, 1),
 (1391006, 1),
 (1391007, 1),
 (1391009, 1),
 (1391010, 1),
 (1391011, 1),
 (1391013, 1),
 (1391030, 1),
 (1391031, 1),
 (1391032, 1),
 (1391043, 1),
 (1391056, 1),
 (1391060, 1),
 (1391066, 1),
 (1391075, 1),
 (1391093, 1),
 (1391106, 1)]

In [53]:
print(mybdesigner.annotation.tabix_intervals('Chr9:1390806-1391207'))

Chr9	1390768	.	GACTTTTATTTACTAAAATGCCGTGTCATATCATTCATATGA	GACTTTTATTTACTAAAATGCCGTCGTCATATCATTCATATGA,TGAGTTTTAGCAACCCGTGGACCAAAACCTACAGTTCGGTTC,AACTTTTATTTACTAAAATACCGTCGTCATATCATTCATATGA,GACTTTTATTTACTAAAATGCCATCGTCATATAATTCATATGA	.	PASS	AR2=1;DR2=1;AF=0.455,0.273,0.182,0.045	GT:DS:GP	3|1:2:0,0,0,0,0,0,0,1,0,0,0,0,0,0,0	1|3:2:0,0,0,0,0,0,0,1,0,0,0,0,0,0,0	1|3:2:0,0,0,0,0,0,0,1,0,0,0,0,0,0,0	1|2:2:0,0,0,0,1,0,0,0,0,0,0,0,0,0,0	4|1:2:0,0,0,0,0,0,0,0,0,0,0,1,0,0,0	1|3:2:0,0,0,0,0,0,0,1,0,0,0,0,0,0,0	1|2:2:0,0,0,0,1,0,0,0,0,0,0,0,0,0,0	0|2:1:0,0,0,1,0,0,0,0,0,0,0,0,0,0,0	1|2:2:0,0,0,0,1,0,0,0,0,0,0,0,0,0,0	1|2:2:0,0,0,0,1,0,0,0,0,0,0,0,0,0,0	2|1:2:0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
Chr9	1390829	.	C	T	.	PASS	AR2=1;DR2=1;AF=0.045	GT:DS:GP	0|0:0:1,0,0	0|0:0:1,0,0	0|1:1:0,1,0	0|0:0:1,0,0	0|0:0:1,0,0	0|0:0:1,0,0	0|0:0:1,0,0	0|0:0:1,0,0	0|0:0:1,0,0	0|0:0:1,0,0	0|0:0:1,0,0
Chr9	1390875	.	A	G	.	PASS	AR2=1;DR2=1;AF=0.091	GT:DS:GP	0|0:0:1,0,0	0|0:0:1,0,0	0|0:0:1,0,0	0|0:0:1,0,0	1|0:1:0,1,0	0|1:1:0,1,0	

In [35]:
print(mybdesigner.annotation.tabix_intervals("Chr9:1391006-1391207"))

Chr9	1391007	.	A	G	.	PASS	AR2=1;DR2=1;AF=0.227	GT:DS:GP	0|0:0:1,0,0	0|0:0:1,0,0	0|0:0:1,0,0	1|0:1:0,1,0	0|0:0:1,0,0	0|0:0:1,0,0	0|1:1:0,1,0	0|1:1:0,1,0	1|0:1:0,1,0	0|0:0:1,0,0	1|0:1:0,1,0
Chr9	1391009	.	G	T,A	.	PASS	AR2=1;DR2=1;AF=0.227,0.182	GT:DS:GP	2|0:1:0,0,0,1,0,0	0|2:1:0,0,0,1,0,0	2|0:1:0,0,0,1,0,0	1|0:1:0,1,0,0,0,0	0|0:0:1,0,0,0,0,0	2|0:1:0,0,0,1,0,0	0|1:1:0,1,0,0,0,0	0|1:1:0,1,0,0,0,0	1|0:1:0,1,0,0,0,0	0|0:0:1,0,0,0,0,0	1|0:1:0,1,0,0,0,0
Chr9	1391010	.	G	A	.	PASS	AR2=1;DR2=1;AF=0.227	GT:DS:GP	0|0:0:1,0,0	0|0:0:1,0,0	0|0:0:1,0,0	1|0:1:0,1,0	0|0:0:1,0,0	0|0:0:1,0,0	0|1:1:0,1,0	0|1:1:0,1,0	1|0:1:0,1,0	0|0:0:1,0,0	1|0:1:0,1,0
Chr9	1391011	.	T	A	.	PASS	AR2=1;DR2=1;AF=0.227	GT:DS:GP	0|0:0:1,0,0	0|0:0:1,0,0	0|0:0:1,0,0	1|0:1:0,1,0	0|0:0:1,0,0	0|0:0:1,0,0	0|1:1:0,1,0	0|1:1:0,1,0	1|0:1:0,1,0	0|0:0:1,0,0	1|0:1:0,1,0
Chr9	1391013	.	GCTTG	ATGGC,TTATA	.	PASS	AR2=0.993;DR2=0.993;AF=0.27,0.002	GT:DS:GP	0|0:0:1,0,0,0,0,0	1|0:0.981:0.019,0.932,0,0.049,0,0	0|0:0:1,0,0,0,0,0	1|0:1:0,1,0,0,0,0	0|0

In [ ]:
mybdesigner=VcfPrimerDesign()

In [ ]:
d.PrimerDesign?

In [ ]:
testPD.reference['k69_93535'][12:30]

In [ ]:
foo=34
bar=67

In [ ]:
testPD.reference['k69_93535'][foo:bar]

In [ ]:
testPD.getseqslice('k69_93535',12,30)